# Databricks Asset Bundles (DABs) - Part 2
## CI/CD with Azure DevOps

In the previous session, we configured a DAB project and deployed it to a **Dev** environment using the CLI and UI.
In this session, we will automate the deployment to a **QA (Quality Assurance)** environment using **Azure DevOps Pipelines**.

### Learning Objectives
1.  Setup permissions for Service Principals in Databricks.
2.  Configure **Azure DevOps Variable Groups** for secure credential management.
3.  Write an **Azure Pipeline YAML** (`azure-pipelines.yml`) for DAB deployment.
4.  Understand how to parameterize pipelines for different environments.
5.  Deploy the bundle to a QA workspace using the Service Principal.

### Prerequisites
*   A **QA Databricks Workspace** (separate from Dev).
*   An **Azure Service Principal (SP)** created and added to the Databricks Account.
*   **Azure DevOps Project** connected to your Git repository.
*   **Parallel Jobs** (Free Tier or Paid) enabled in Azure DevOps to run pipelines.

## 1. Security & Permissions Setup

Before automating deployment, we need to ensure the Service Principal (SP) has the right access. We do not deploy to QA/Prod using user credentials; we use a Service Principal.

### Step 1: Workspace Access
1.  Go to **Databricks Account Console** -> **Workspaces** -> Select **QA Workspace**.
2.  Go to **Permissions** -> **Add permissions**.
3.  Search for your **Service Principal** (e.g., `azure-sp`).
4.  Grant **Admin** role (required to create jobs/pipelines/schemas).

### Step 2: Catalog Permissions
1.  Log in to the **QA Workspace**.
2.  Go to **Catalog Explorer**.
3.  Select the target catalog (e.g., `qa`).
4.  Click **Permissions** -> **Grant**.
5.  Grant **ALL PRIVILEGES** to the Service Principal. This allows the SP to create the `bronze` schema defined in our bundle.

## 2. Azure DevOps Variable Groups

To securely pass secrets (Client ID, Client Secret, Host URL) to the pipeline without hardcoding them in the repository, we use **Variable Groups**.

1.  Go to **Azure DevOps** -> **Pipelines** -> **Library**.
2.  Click **+ Variable group**.
3.  Name it: `qa_variables` (Naming convention is important for dynamic referencing).
4.  Add the following variables:
    *   `DATABRICKS_HOST`: URL of your QA workspace (e.g., `https://adb-xxxx.net`).
    *   `CLIENT_ID`: Application (client) ID of your Service Principal.
    *   `CLIENT_SECRET`: Client Secret of your Service Principal (Click the 🔒 icon to secure it).
5.  **Save** the group.

*Note: For Production, create another group named `prod_variables` with Production credentials.*

## 3. Updating `databricks.yml` for QA

We need to ensure our bundle configuration knows about the QA environment.

Update `databricks.yml`:

```yaml
targets:
  dev:
    # ... (dev config)
  
  qa:
    workspace:
      host: https://adb-<your-qa-workspace-url>.net
    # We don't set 'mode: development' here, so it acts as a production deployment
    # It will deploy artifacts to the workspace (not source-linked)
```

## 4. Creating the Azure Pipeline YAML

Create a file named `azure-pipelines.yml` in the root of your repository.

### Key Components of the Pipeline:
1.  **Trigger**: Set to `none` for manual triggering (can be changed to `main` for automated CI).
2.  **Parameters**: To allow selecting Environment (`env`) and Catalog (`catalog`) at runtime.
3.  **Variables**: Dynamically load the Variable Group based on the selected environment (`${{ parameters.env }}_variables`).
4.  **Steps**:
    *   Install Databricks CLI.
    *   Setup Authentication Profile.
    *   Validate, Summarize, and Deploy Bundle.

### The Code (`azure-pipelines.yml`)

```yaml
# Azure Pipeline for DABs Deployment
trigger: none

pool:
  vmImage: ubuntu-latest

parameters:
  - name: env
    displayName: Environment
    type: string
    default: qa
    values:
      - dev
      - qa
  
  - name: catalog
    displayName: Target Catalog Name
    type: string
    default: qa

variables:
  - group: ${{ parameters.env }}_variables

stages:
  - stage: onRelease
    jobs:
      - job: ReleaseJob
        displayName: Deploy Databricks Asset Bundle
        steps:
          - checkout: self
            displayName: Checkout Source Code

          # Step 1: Install Databricks CLI
          - script: |
              curl -fsSL https://raw.githubusercontent.com/databricks/setup-cli/main/install.sh | sh
            displayName: 'Install Databricks CLI'

          # Step 2: Authenticate (M2M Profile)
          # We create a .databrickscfg file with the secrets from Variable Group
          - script: |
              echo "[azure-sp]" >> ~/.databrickscfg
              echo "host = $(DATABRICKS_HOST)" >> ~/.databrickscfg
              echo "client_id = $(CLIENT_ID)" >> ~/.databrickscfg
              echo "client_secret = $(CLIENT_SECRET)" >> ~/.databrickscfg
            displayName: 'Setup Authentication Profile'

          # Step 3: Validate Profile
          - script: |
              databricks auth profiles
            displayName: 'List Auth Profiles'

          # Step 4: Validate Bundle
          # We pass the profile 'azure-sp' and override the catalog variable dynamically
          - script: |
              databricks bundle validate --target ${{ parameters.env }} --profile azure-sp --var "catalog=${{ parameters.catalog }}"
            displayName: 'Validate Bundle Configuration'

          # Step 5: Deploy Bundle
          - script: |
              databricks bundle deploy --target ${{ parameters.env }} --profile azure-sp --var "catalog=${{ parameters.catalog }}"
            displayName: 'Deploy Bundle to ${{ parameters.env }}'
```

## 5. Executing the Deployment

1.  **Push Code**: Commit and push the `azure-pipelines.yml` and updated `databricks.yml` to your repository.
2.  **Create Pipeline**:
    *   In Azure DevOps, go to **Pipelines** -> **New Pipeline**.
    *   Select **Azure Repos Git** -> Your Repository.
    *   Select **Existing Azure Pipelines YAML file** and point to `azure-pipelines.yml`.
3.  **Run Pipeline**:
    *   Click **Run Pipeline**.
    *   Select `qa` for Environment and type `qa` for Catalog.
    *   **Approve Permission**: The first time you run, you will be asked to permit access to the `qa_variables` group. Click **Permit**.

### What happens during deployment?
Unlike the Dev deployment (Source-Linked), deploying to QA via this pipeline does the following:
1.  Uploads all source files (notebooks, YAMLs) to a hidden location in the QA Workspace (`/Users/<sp-id>/.bundle/...`).
2.  Creates/Updates the Jobs and Pipelines in QA to point to these uploaded files.
3.  Creates the Schema defined in the resources (if it doesn't exist).

In [ ]:
# Optional: Validation Script (Local)
# Before pushing to Azure DevOps, you can validate your changes locally using the CLI.
# Note: You need the profile configured locally or use your user credentials.

# !databricks bundle validate -t dev

## 6. Summary

We have successfully built a complete CI/CD workflow for Databricks:

| Environment | Deployment Method | Credential | Artifact Location |
| :--- | :--- | :--- | :--- |
| **Dev** | CLI / UI (Manual) | User (You) | Workspace Repo (Source-Linked) |
| **QA** | Azure Pipeline (Automated) | Service Principal | `.bundle` Folder (Isolated Copy) |

This ensures that developers can iterate fast in Dev without affecting others, while QA/Prod deployments are stable, isolated, and managed via version control and automation.